In [1]:
using QuantumOptics
using PyPlot
using NPZ
using ProgressBars
using LinearAlgebra
using SparseArrays

In [2]:
const ħ = 6.626e-34 / 2π
const m_p = 1.67262192e-27

1.67262192e-27

In [3]:
function H_RydbergCZ(Ω, ϕ; δ=0.0, δ_Doppler=0.0, δ_blockade=Inf, λ=302e-9, ω_trap=2π * 30e-3, nMax=10)
    n_atom = 4
    m_basis = FockBasis(nMax)
    a_basis = NLevelBasis(n_atom)

    # Lamb-Dicke
    η = 2π / λ * sqrt(ħ / 2 / 171 / m_p / ω_trap)
    kick_pos = exp(im * η * (create(m_basis) + destroy(m_basis)))
    kick_neg = exp(-im * η * (create(m_basis) + destroy(m_basis)))

    # Free space Hamiltonian (trap turned off during gate), p^2/2m
    H0 = -ω_trap / 4 * (tensor(identityoperator(a_basis), identityoperator(a_basis),
                               (create(m_basis) - destroy(m_basis))^2, identityoperator(m_basis))
                        + tensor(identityoperator(a_basis), identityoperator(a_basis),
                                 identityoperator(m_basis), (create(m_basis) - destroy(m_basis))^2))
    # Rydberg Rabi
    H =  Ω/2*(cis(-ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 2))',
                    identityoperator(m_basis), kick_neg
                )   # 01 -> 0r
                + cis(ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 3))',
                    identityoperator(m_basis), kick_pos
                )  # 0r -> 01
                + cis(-ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 1)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 1))',
                    kick_neg, identityoperator(m_basis)
                )# 10 -> r0
                + cis(ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 1)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 1))',
                    kick_pos, identityoperator(m_basis)
                )# r0 -> 10
                + cis(-ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 2))',
                    identityoperator(m_basis), kick_neg
                )# 11 -> 1r
                + cis(ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3))',
                    identityoperator(m_basis), kick_pos
                )# 1r -> 11
                + cis(-ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 2))',
                    kick_neg, identityoperator(m_basis)
                )# 11 -> r1
                + cis(ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2))',
                    kick_pos, identityoperator(m_basis)
                )
            ) # r1 -> 11
    
    # Finite blockade
    if isfinite(δ_blockade)
        H += δ_blockade*tensor(
            tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3))',
            identityoperator(m_basis), identityoperator(m_basis)
        )
        H += Ω/2*(cis(-ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3))',
                    kick_neg, identityoperator(m_basis)
                )# 1r -> rr
                + cis(ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3))',
                    kick_pos, identityoperator(m_basis)
                )# rr -> 1r
                + cis(-ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2))',
                    identityoperator(m_basis), kick_neg
                )# r1 -> rr
                + cis(ϕ)*tensor(
                    tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3))',
                    identityoperator(m_basis), kick_pos
                )     # rr -> r1
            )
    end

    # Opposite Doppler shift for two atoms for drive-by gates
    if δ_Doppler != 0
        H += δ_Doppler/2*tensor(
            (-tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 3))' # -0r
             -tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3))'  # -1r
             +tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 1)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 1))'  # +r0
             +tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2))'), # +r1
            identityoperator(m_basis), identityoperator(m_basis)
        )
    end
    
    # Laser detuning
    if δ != 0
        H += δ*tensor(
            (tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 3))'     # 0r
             +tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 3))'     # 1r
             +tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 1)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 1))'     # r0
             +tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 2))'     # r1
             +2*tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3)) ⊗ tensor(nlevelstate(a_basis, 3), nlevelstate(a_basis, 3))'),   # rr
            identityoperator(m_basis), identityoperator(m_basis)
        )
    end

    return H + H0
end

H_RydbergCZ (generic function with 1 method)

In [4]:
# The `thermalstate` function from QuantumOptics doesn't work with zero temperature
function thermal_fock(basis::FockBasis, nbar)
    nmax = length(basis) - 1
    ns = 0:nmax
    α = nbar / (1 + nbar)
    return Operator(basis, diagm((1 - α) .* α.^ns))
end

thermal_fock (generic function with 1 method)

In [5]:
function Rydberg_BellFid(ϕ_list, Ω, t_gate, num_slices;
                         δ=0, δ_Doppler=0, δ_blockade=Inf, γ=0, γ_loss=0,
                         λ=302e-9, ω_trap=2π * 30e-3, n_init=0, nMax=10)
    n_atom = 4
    m_basis = FockBasis(nMax)
    a_basis = NLevelBasis(n_atom)
    basis = a_basis ⊗ a_basis ⊗ m_basis ⊗ m_basis

    @assert length(ϕ_list) == num_slices + 1
    dt = t_gate / num_slices
    c_ops = Operator{typeof(basis),typeof(basis),SparseMatrixCSC{ComplexF64,Int}}[]
    # initial state |00> + |01> + |10> + |11>
    state = tensor(
        normalize(dm(tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 1))
        +tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 2))
        +tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 1))
        +tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 2)))),
        thermal_fock(m_basis, n_init),
        thermal_fock(m_basis, n_init)
    )
    # target state |00> + |01> + |10> - |11>
    target = normalize(tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 1))
            +tensor(nlevelstate(a_basis, 1), nlevelstate(a_basis, 2))
            +tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 1))
            -tensor(nlevelstate(a_basis, 2), nlevelstate(a_basis, 2)))
    for ϕ in tqdm(ϕ_list[1:end - 1])
        H = H_RydbergCZ(
                Ω, 
                ϕ,
                δ=δ,
                δ_Doppler=δ_Doppler,
                δ_blockade=δ_blockade,
                λ=λ,
                ω_trap=ω_trap,
                nMax=nMax
        )
        state = timeevolution.master([0, dt], state, H, c_ops)[2][end]
    end
    state = ptrace(state, [3, 4])
        
    Zrot = tensor(Operator(a_basis, diagm([1, cis(ϕ_list[end]), 1, 1])), 
                  Operator(a_basis, diagm([1, cis(ϕ_list[end]), 1, 1])))
    state = Zrot * state * dagger(Zrot)
    return abs2(fidelity(state, dm(target)))
end

Rydberg_BellFid (generic function with 1 method)

In [6]:
pulse_base = npzread("seq_base.npz")
phase_list = pulse_base["phase_list"]
num_slices = length(phase_list) - 1
Omega = pulse_base["Omega"]
t_gate = pulse_base["t_gate"]

Rydberg_BellFid(phase_list,
                Omega,
                t_gate,
                num_slices,
                δ=0.,
                δ_Doppler=0,
                δ_blockade=Inf,
                γ=0,
                γ_loss=0,
                λ=302e-9,
                ω_trap=2π * 30e-3,
                n_init=0,
                nMax=14
               )

0.0%┣                                                ┫ 0/8 [00:01<00:-5, -1s/it]
12.5%┣█████▍                                     ┫ 1/8 [00:21<Inf:Inf, InfGs/it]
25.0%┣███████████▊                                   ┫ 2/8 [00:35<03:29, 35s/it]
37.5%┣█████████████████▋                             ┫ 3/8 [00:49<02:03, 25s/it]
50.0%┣███████████████████████▌                       ┫ 4/8 [01:03<01:23, 21s/it]
62.5%┣█████████████████████████████▍                 ┫ 5/8 [01:14<00:55, 18s/it]
75.0%┣███████████████████████████████████▎           ┫ 6/8 [01:28<00:35, 18s/it]
87.5%┣█████████████████████████████████████████▏     ┫ 7/8 [01:44<00:17, 17s/it]
100.0%┣██████████████████████████████████████████████┫ 8/8 [01:59<00:00, 17s/it]
100.0%┣██████████████████████████████████████████████┫ 8/8 [01:59<00:00, 17s/it]


0.9971040978587192

In [7]:
# Doppler shift
pulse_Doppler = npzread("seq_Doppler.npz")
phase_list = pulse_Doppler["phase_list"]
num_slices = length(phase_list) - 1
det_Dopp = pulse_Doppler["det_Dopp"]
Omega = pulse_Doppler["Omega"]
t_gate = pulse_Doppler["t_gate"]

Rydberg_BellFid(phase_list,
                Omega,
                t_gate,
                num_slices,
                δ=0.,
                δ_Doppler=det_Dopp, 
                δ_blockade=Inf,
                γ=0,
                γ_loss=0,
                ω_trap=2π * 30e-3,
                nMax=10
               )

0.0%┣                                               ┫ 0/10 [00:00<00:00, -0s/it]
10.0%┣████▏                                     ┫ 1/10 [01:13<Inf:Inf, InfGs/it]
20.0%┣█████████                                    ┫ 2/10 [01:49<14:36, 109s/it]
30.0%┣█████████████▉                                ┫ 3/10 [02:27<08:33, 73s/it]
40.0%┣██████████████████▍                           ┫ 4/10 [03:06<06:12, 62s/it]
50.0%┣███████████████████████                       ┫ 5/10 [03:48<04:45, 57s/it]
60.0%┣███████████████████████████▋                  ┫ 6/10 [04:35<03:40, 55s/it]
70.0%┣████████████████████████████████▏             ┫ 7/10 [05:19<02:39, 53s/it]
80.0%┣████████████████████████████████████▉         ┫ 8/10 [06:00<01:43, 51s/it]
90.0%┣█████████████████████████████████████████▍    ┫ 9/10 [06:40<00:50, 50s/it]
100.0%┣████████████████████████████████████████████┫ 10/10 [07:54<00:00, 53s/it]
100.0%┣████████████████████████████████████████████┫ 10/10 [07:54<00:00, 53s/it]


0.9925141140918055

In [8]:
# Finite blockade

pulse_blockade = npzread("seq_blockade.npz")
phase_list = pulse_blockade["phase_list"]
num_slices = length(phase_list) - 1
det_blockade = pulse_blockade["det_blockade"]
Omega = pulse_blockade["Omega"]
t_gate = pulse_blockade["t_gate"]

Rydberg_BellFid(phase_list, 
                Omega, 
                t_gate, 
                num_slices,
                δ=0., 
                δ_Doppler=0, 
                δ_blockade=det_blockade[],
                γ=0,
                γ_loss=0,
                ω_trap=2π * 30e-3,
                nMax=10
               )

0.0%┣                                              ┫ 0/100 [00:00<00:00, -0s/it]
1.0%┣▍                                         ┫ 1/100 [00:38<Inf:Inf, InfGs/it]
2.0%┣▉                                          ┫ 2/100 [01:07<01:49:27, 67s/it]
3.0%┣█▎                                         ┫ 3/100 [01:30<01:12:29, 45s/it]
4.0%┣█▊                                         ┫ 4/100 [01:55<01:01:23, 38s/it]
5.0%┣██▎                                           ┫ 5/100 [02:30<59:30, 38s/it]
6.0%┣██▊                                           ┫ 6/100 [03:04<57:32, 37s/it]
7.0%┣███▏                                          ┫ 7/100 [03:24<52:38, 34s/it]
8.0%┣███▊                                          ┫ 8/100 [03:49<50:11, 33s/it]
9.0%┣████▏                                         ┫ 9/100 [04:20<49:18, 33s/it]
10.0%┣████▍                                       ┫ 10/100 [04:42<46:55, 31s/it]
11.0%┣████▉                                       ┫ 11/100 [05:10<46:00, 31s/it]
12.0%┣█████▎                

0.9966998622612087